In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation, AveragePooling2D
import tensorflow as tf

In [2]:
import os
classes = os.listdir("data")
print(classes)

['angry', 'happy', 'laughing', 'nerd', 'sad', 'sick', 'surprised', 'yawning']


In [3]:
model = Sequential()
# input is 150x150x3
model.add(Conv2D(128, (11, 11), input_shape=(300, 300, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(classes), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
# clear up images so if it cannot open remove it
import os
from PIL import Image

for i in classes:
    for j in os.listdir("data/"+i):
        try:
            Image.open("data/"+i+"/"+j)
        except:
            os.remove("data/"+i+"/"+j)

In [5]:
# load data where each folder has a bunch of images and the label is the folder name
import numpy as np
from PIL import Image

data = []
labels = []

for folder in classes:
    print(folder)
    images = os.listdir("data/" + folder)
    for image in images:
        try:
            img = Image.open("data/" + folder + "/" + image)
            # remove alpha channel
            img = img.convert('RGB')
            img = img.resize((300, 300))
            img = np.array(img)
            data.append(img)
            label = np.zeros(len(classes))
            label[classes.index(folder)] = 1
            labels.append(label)
        except:
            print("Error loading image")

angry
happy
laughing
nerd


C:\Users\adoma\miniconda3\envs\tf\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


sad
sick
surprised
Error loading image
yawning


In [6]:
data = np.array(data)
labels = np.array(labels)

In [7]:
print(data.shape)
print(labels.shape)

(1788, 300, 300, 3)
(1788, 8)


In [8]:
# split data into training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [9]:
# train model
model.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
23/23 [==============================] - 37s 790ms/step - loss: 28.1688 - accuracy: 0.1385 - val_loss: 2.0640 - val_accuracy: 0.1788
Epoch 2/20
23/23 [==============================] - 11s 500ms/step - loss: 2.0480 - accuracy: 0.1860 - val_loss: 2.1116 - val_accuracy: 0.1173
Epoch 3/20
23/23 [==============================] - 11s 484ms/step - loss: 2.0362 - accuracy: 0.2063 - val_loss: 2.0141 - val_accuracy: 0.2123
Epoch 4/20
23/23 [==============================] - 11s 487ms/step - loss: 1.8990 - accuracy: 0.3077 - val_loss: 1.8894 - val_accuracy: 0.3017
Epoch 5/20
23/23 [==============================] - 11s 491ms/step - loss: 1.6562 - accuracy: 0.4126 - val_loss: 1.6702 - val_accuracy: 0.3883
Epoch 6/20
23/23 [==============================] - 11s 493ms/step - loss: 1.5719 - accuracy: 0.4441 - val_loss: 1.6688 - val_accuracy: 0.3631
Epoch 7/20
23/23 [==============================] - 11s 495ms/step - loss: 1.2583 - accuracy: 0.5650 - val_loss: 1.3740 - val_accuracy: 0.569

In [11]:
model.save("model_20.h5")